In [ ]:
import requests

In [ ]:
my_key="RGAPI-3cb6bf63-086f-47d8-9f48-34853a8e8291"

In [ ]:
import pandas as pd

###Use Beautiful Soup to get Best Player ID

In [ ]:
import requests
from bs4 import BeautifulSoup

# Define the URLs
urls = [
    "https://lolprofile.net/leaderboards/na/0",
    "https://lolprofile.net/leaderboards/na/1",
    "https://lolprofile.net/leaderboards/na/2",
    "https://lolprofile.net/leaderboards/na/3",
    "https://lolprofile.net/leaderboards/na/4",
    "https://lolprofile.net/leaderboards/na/5"
]

# Create an empty list to store all span elements
text_contents = [] 

# Iterate over the URLs
for url in urls:
    # Make the request and parse the HTML
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Extract the desired elements using BeautifulSoup selectors
    span_elements = soup.select('tbody tr td:not([class]) span')
    
    # Add the span elements to the combined list
    text_contents.extend(span.text for span in span_elements)

# Iterate over the combined list of span elements and print their contents

print(text_contents)




In [ ]:
len(text_contents)

In [ ]:
len(set(text_contents))

In [ ]:
id = list(set(text_contents))

In [ ]:
id

In [ ]:
len(id)

## Use Player ID to get PUUID

In [ ]:
def get_puuid(my_key,id):
   puuid_list=[]
   for i in range(len(id)):
     try:
      summoner_url = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + str(id[i]) + '?api_key=' + my_key
      get_player_info = requests.get(summoner_url).json()
      puuid = get_player_info['puuid']
      puuid_list.append(puuid)
     except KeyError:
      continue

   return puuid_list
     

In [ ]:
puuid=get_puuid(my_key,id)

In [ ]:
len(puuid)

In [ ]:
puuid

## Use PUUID to get Match Id

In [ ]:
def get_match_id(puuid):
    match_id_list=[]
    for i in range(len(puuid)):
      get_match= "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/" +str(puuid[i])+ "/ids?type=ranked&start=0&count=3" + '&api_key=' + my_key
      match_info = requests.get(get_match).json()
      match_id_list.append(match_info)

    return match_id_list

In [ ]:
match_id=get_match_id(puuid)

In [ ]:
len(match_id)

In [ ]:
match_id

In [ ]:
combined_match_id=[]
for i in match_id:
    combined_match_id += i
len(combined_match_id)

## Remove Duplicate in match_id in case they are in the same game

In [ ]:
final_combined_match_id = list(set(combined_match_id))

In [ ]:
len(final_combined_match_id)

## Get match info with match-id

In [ ]:
def get_game_info(final_combined_match_id):
  mydf_list=[]
  for i in final_combined_match_id[:10]:
    get_in_game_info_="https://americas.api.riotgames.com/lol/match/v5/matches/" +str(i)+ '?api_key=' + my_key 
    In_game_info = requests.get(get_in_game_info_).json()
    In_game_info=In_game_info["info"]["participants"] 
    mydf_list.append(pd.DataFrame.from_dict(In_game_info))
  df1=pd.concat(mydf_list)
  return df1


In [ ]:
list1=get_game_info(final_combined_match_id)
    

In [ ]:
list1.shape

In [ ]:
list1.head()